In [1]:
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel,RunnablePassthrough

from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_core.output_parsers import StrOutputParser

from langchain_openai import OpenAIEmbeddings
import streamlit as st
 
import os


In [2]:
from dotenv import load_dotenv
load_dotenv()
token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "text-embedding-3-small"

In [3]:
#used Hugging Face Embeddings
# embeddings = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')

# client to take off the embedding model
embedddings  = OpenAIEmbeddings(
    model = 'text-embedding-3-large',
    base_url =endpoint,
    api_key = token,

)

In [4]:
#CLIENT to take care of language model , ChatOpenAI: Initializes a ChatOpenAI model with specific parameters
# This model is used to generate responses based on user queries.

model = ChatOpenAI(
    base_url = "http://localhost:5272/v1/",
    api_key="ai-df",
    model = "Phi-3-mini-128k-cpu-int4-rtn-block-32-acc-level-4-onnx",
    temperature= 0.7
    
)

In [5]:
#  This line initializes a Chroma vector store by loading it from the specified directory.
load_db = Chroma(persist_directory='./ai-tkt-dir',embedding_function=embedddings)

# Convert to Retriever:Converts the Chroma vector store into a retriever object.

retreiver = load_db.as_retriever(search_kwargs={'k':3})

# k - search_kwargs={'k': 3}: Specifies the search parameters for the retriever. In this case, k=3 means that the retriever 
# will return the top 3 most similar documents for any given query.

/tmp/ipykernel_372523/2169804551.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  load_db = Chroma(persist_directory='./ai-tkt-dir',embedding_function=embedddings)


<!-- 
Introduction: Sets the context for the AI assistant.
Relevance: Ensures responses are relevant to the toolkit and the user's query.
Avoid Irrelevant Information: Instructs the AI to avoid unrelated information.
Professional Tone: Ensures responses are professional, accurate, and helpful.
Adhere to User's Question: Instructs the AI to focus on the user's question.
Unknown Answers: Provides guidance on how to respond if the AI does not know the answer. -->

In [6]:
template = """ You are a specialized AI assistant for the Microsoft Visual Studio Code AI Toolkit.\n
    Your responses should be strictly relevant to this product and the user's query. \n
    Avoid providing information that is not directly related to the toolkit.
    Maintain a professional tone and ensure your responses are accurate and helpful.
    Strictly adhere to the user's question and provide relevant information. 
    If you do not know the answer then respond "I dont know".Do not refer to your knowledge base.
    {context}
    Question:
    {question}
"""



In [7]:
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

# RunnableParallel object is created 
setup_and_retrieval = RunnableParallel(
	    {"context": retreiver, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

In [8]:
query=input("Enter your query:")
resp=chain.invoke(query)
print(resp)

Number of requested results 3 is greater than number of elements in index 2, updating n_results = 2


 The VS AI Toolkit is an extension for Visual Studio Code. It provides features to download, test, fine-tune, and deploy AI models. You can use it either within your applications or in the cloud.
